## Criação do Streaming de Dados


Usando a API de Streaming do Twitter com o Tweepy são coletados os tweets de um assunto escolhido (Saint Patrick's Day). Um servidor conectado na porta 5556 gera um stream de dados que é coletado pelo SparkStreaming a cada 10 segundos e consolidado a cada 20 segundos. Foram escolhidos como atributos para análise o texto do tweet e os horários agrupados por Hora e Minutos

In [2]:
ssc  = StreamingContext (sc, 10 )
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("localhost", 5556)
lines = socket_stream.window( 20 )

( lines.map(lambda x: json.loads(x)).
  map(lambda x: {"text" : x["text"], "created_at": parse(x["created_at"]).strftime("%H:%M"), "_id" : x["id"]})
  .foreachRDD( lambda rdd: rdd.toDF().fillna('').registerTempTable("tweets") ) )

ssc.start()
start_stream = time.time()

## Análise, Visualização e Armazenamento dos Dados

A cada 20 segundos os dados são processados, sendo que são realizadas as seguintes tarefas:
* Salvar Dados no MongoDB
* Criar função Map utilizando Horas:Minutos e reduce para obter a quantidade de tweets em cada momento
* Pré Processar o Texto e gerar tokens de palavras que são utilizadas em outro Map/Reduce para obter o número de ocorrência das palavras
* Gerar o gráfico da quantidade histórica de tweets e de palavras mais utilizadas

Isso é repetido até atingir um período de tempo (no caso 10 Minutos)


In [ ]:
counts = []
words = []

now = time.time()

time.sleep(20 - (now - start_stream))

while time.time() - start_stream <= 600:
    start = time.time()
    tweet = sqlContext.sql('Select * from tweets')
    tweet.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()
    
    ## Análises e Plot dos Dados
    
    end = time.time()
    sleep = 20 - (end - start)
    time.sleep( sleep if sleep >= 0 else 0)
ssc.stop()

As análises geradas após 10 minutos são mostradas na imagem a seguir, que mostra as 10 palavras mais utilizadas e a quantidade e a série temporal do número de tweets

![Número de Palavras e de Tweets](output_5_0.png)

A estrutura dos dados salvos no MongoDB é mostrada a seguir, tendo como o total de dados inseridos 5234 tweets

![](mongo.png)

### Tabela de Número de Tweets

In [15]:
pd.DataFrame(counts, columns = ["Horário", "Número de Tweets"]).sort_values("Horário")

,Horário,Número de Tweets
9,17:22,245
7,17:23,523
2,17:24,549
3,17:25,571
4,17:26,535
6,17:27,534
10,17:28,547
0,17:29,546
8,17:30,641
1,17:31,590


### Tabela de Número de Palavras

In [17]:
pd.DataFrame(words, columns = ["Palavra", "Número de Ocorrências"]).sort_values("Número de Ocorrências", ascending = False).head(20)

,Palavra,Número de Ocorrências
5148,happy,2108
6960,green,1317
0,irish,668
4897,patrick,518
868,look,515
2536,river,470
6022,dyed,465
98,today,403
5851,patricks,340
2187,luck,299
